## Run SparkSession

In [ ]:
import os
import sys

In [ ]:
sys.path.insert(0, "Full\path\to\Hadoop\spark-2.4.7-bin-hadoop2.7\python")
sys.path.insert(0, "Full\path\to\Hadoop\spark-2.4.7-bin-hadoop2.7\python\lib\py4j-0.10.7-src.zip")

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [ ]:
conf = SparkConf()
conf.setMaster("local").setAppName("Graph")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

In [ ]:
spark

### Import modules

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as f
from func import *

### Load data

In [ ]:
%%time
schema = StructType([
        StructField('Payer', StringType()),
        StructField('Recipient', StringType()),
        StructField('Amount', FloatType()),
        StructField('Date', StringType()),
                    ])
df = spark.read.csv('data.csv',schema,';',header=False).repartition(4)

### Change date string column to timestamp

In [ ]:
df = df.withColumn('Date', f.to_timestamp(f.col('Date'), "yyyy-MM-dd HH:mm:ss"))

In [ ]:
df.show(5)

### Data preprocessing

In [ ]:
init_graph = Graph_miner(df, "Payer", "Recipient", "Amount", "Date")

In [ ]:
init_graph.result.head(10)

### Draw graphs

In [ ]:
paint = Painter(init_graph)

In [ ]:
paint.draw(filename = "start_graph")

In [ ]:
paint.filtering_df(r_count_tresh=1)
paint.draw(filename = "r_count_tresh_1")

In [ ]:
paint.filtering_df(sum_tresh = 3000, r_count_tresh=1)
paint.draw(filename = "sum_tresh_3000_r_count_tresh_1")

In [ ]:
paint.filtering_df(sum_tresh = 3000, r_count_tresh=1, acc_name="account_96")
paint.draw(filename = "sum_tresh_3000_r_count_tresh_1_name_account_96")

## Stop SparkSession

In [ ]:
spark.stop()